# N-gram testing for WA
This notebook will be used for diagnosis of incorrect intent detection problem.

<div class="alert alert-block alert-info">
<b>Notebook Summary</b>
<br>

1. <b>Connect to WA</b>: creating a connection to the workspace that we want to use for diagnosis. <br>
2. <b>Setting the data</b>: what phrase do we need to analyse. <br>
3. <b>Creating uni/bi/trigrams combination</b> <br>
4. <b>Send the combinations</b> created in the previous step to the workspace <br>
5. <b>Analyse the results</b>. 
</div>


### What is an n-gram: 
N-gram is a contiguous sequence of n items from a given sample of text

### Scope
This notebook will be used for diagnosis of incorrect intent detection problem. 
We want to use:
- unigrams 
- bigrams 
- trigrams 
to do so. 

The idea is that each intent is sensitive to particular words/combination of words. The scope is to find a fast way to detect why, for instance, a testing phrase didn't trigger the right intent. 

This notebook will help the user understanding why which words and in which positions are the most important in the intent detection phase. 



## Libraries 

In [ ]:
!pip install msgpack
#!pip install watson-developer-cloud
!pip install --upgrade "ibm-watson>=5.3.0"
import json 
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
# you need to download and install nltk in order to use it 
#!pip install nltk 
!pip install nltk 
import nltk
#nltk.download()
nltk.download('punkt')
from nltk.tokenize import sent_tokenize, word_tokenize
from ibm_watson import AssistantV1
from IPython.display import display

## 1. Creating the Watson Assistant object 

<div class="alert alert-block alert-warning">

<b>Data needed from the user</b>
<br>

    
1. <b>Credentials to connect to WA</b> : Log in to IBM cloud, Go to your resource list, select your WA service, and Click Show Credentials - based on what credentials you have, choose option1 (IAM) or option2 (Basic) to connect to your WA instance.<br>
2. <b>Workspace</b> : the ID of the workspace that you want to analyse. - Go to your WA instance, "Skills" tab, click on the three dots of the workspace/skill that you are interested, choose "View API details" and you will find the Workspace ID
</div>

**IAM token**: With some service instances, you authenticate to the API by using **Identity and Access Management (IAM)** - (IBM Cloud is migrating to token-based IAM authentication). You will need `API Key`, `url` and `workpace (skill Id)`

Sign up for the Watson Assistant service and enter your credentials. 

1. Sign up for [Watson Assistant](https://cloud.ibm.com/catalog/services/conversation) in IBM Cloud.
1. On your Watson Assistant service page, click **Launch Watson Assistant**. The Assistant and skills appear in a separate tab.
1. Select your skill which simulates your environment, Click on the 3 dots menu to select **View API Details** option. 
1. On your skill settings page, you should find your skill ID (worksapce) and API Key.
1. Your Service URL should be the root of the workspace URL. The URL for Dallas/US South region looks like this https://api.us-south.assistant.watson.cloud.ibm.com/instances/instanceID
1. Add your skill ID (workspace), APIKey, URL to the next cell and run the cell.

In [ ]:
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
iam_apikey=''  #USER INPUT
url= ''  #USER INPUT

#Frankfurt https://api.eu-de.assistant.watson.cloud.ibm.com/instances/instanceID
#Dallas https://api.us-south.assistant.watson.cloud.ibm.com/instances/instanceID
#London https://api.eu-gb.assistant.watson.cloud.ibm.com/instances/instanceID
#Sydney https://api.au-syd.assistant.watson.cloud.ibm.com/instances/instanceID
#Washington DC https://api.us-east.assistant.watson.cloud.ibm.com/instances/instanceID
#Tokyo https://api.jp-tok.assistant.watson.cloud.ibm.com/instances/instanceID
#Seoul https://api.kr-seo.assistant.watson.cloud.ibm.com/instances/instanceID

authenticator = IAMAuthenticator(iam_apikey)
assistant = AssistantV1(
    version='2021-06-14',
    authenticator=authenticator
)
assistant.set_service_url(url)
  
workspace = '' #USER INPUT
userid= 'usertest' #USER INPUT

### Let's try and see if it's connected to the workspace correctly 

In [ ]:

response = assistant.message(workspace_id=workspace, input={'text': 'Hello'}, alternate_intents= True, user_id= userid)
dumps = json.dumps(response.get_result(), indent=2)
print(dumps)

data = json.loads(dumps)
intent= data['intents'][0]['intent']
confidence = data['intents'][0]['confidence']
print("intent detected: ", intent, "and the confidence level of the intent is :", confidence)
print('success')

## 2. Set the data that you want to analyse

<div class="alert alert-block alert-warning">

<b>Data needed from the user</b>
<br>

1. <b>Data</b> : the sentence that you want to analyse<br>
2. <b>Target</b> : it is the target intent (the intent the we predict should be triggered by the test phrase)<br>
2. <b>Threshold</b>: this is the minimum level of confidence level that you are expecting in order to check if the input text is triggering the right intent. It will be used in the visualization of the results. 
</div>


In [ ]:
data = "tell me what to do before ordering a pizza"
target = "order_pizza" 
threshold = 0.7

## 3. Creating Uni/Bi/Tri- grams 

In [ ]:
def create_unigram_set(data, target):
    """
    This is the unigram function: it creates all the possible combination of (phrase -(minus) unigram).  
    """
    words = word_tokenize(data)
    combination = pd.DataFrame({'phrase':data,'target': target}, index=[0])
    
    #temp_array.append(words)
    for i in range(len(words)):
        words_temp = words.copy()
        temp_array = []
        taken = words_temp[i]
        del words_temp[i]
        temp_array.append(words_temp)
        
        #concatate words into a complete sentence to send to WA
        for j in range(len(temp_array)):
            temp_array[j] = " ".join(temp_array[j])
            combination = combination.append({'phrase':temp_array[j],\
                                              'target': target, \
                                              'taken': taken}, ignore_index=True)
    return combination

def create_bigram_set(data, target):
    """
    This is the bigram function: it creates all the possible combination of (phrase -(minus) bigram).
    """
    words = word_tokenize(data)
    combination = pd.DataFrame({'phrase':data,'target': target}, index=[0])
    
    #temp_array.append(words)
    for i in range(len(words)-1):
        words_temp = words.copy()
        temp_array = []
        taken = words_temp[i]+ "-" + words_temp[i+1]
        del_index=[i,i+1]
        words_temp = [x for e, x in enumerate(words_temp) if e not in del_index]
        temp_array.append(words_temp)
        
        #concatate words into a complete sentence to send to WA
        for j in range(len(temp_array)):
            temp_array[j] = " ".join(temp_array[j])
            combination = combination.append({'phrase':temp_array[j],\
                                              'target': target, \
                                              'taken': taken}, ignore_index=True)
    return combination

def create_trigram_set(data, target):
    """
    This is the trigram function: it creates all the possible combination of (phrase -(minus) trigram).
    """
    words = word_tokenize(data)
    combination = pd.DataFrame({'phrase':data,'target': target}, index=[0])
    
    #temp_array.append(words)
    for i in range(len(words)-2):
        words_temp = words.copy()
        temp_array = []
        taken = words_temp[i]+ "-" + words_temp[i+1] + "-" + words_temp[i+2]
        del_index=[i, i+1, i+2]
        words_temp = [x for e, x in enumerate(words_temp) if e not in del_index]
        temp_array.append(words_temp)
        
        #concatate words into a complete sentence to send to WA
        for j in range(len(temp_array)):
            temp_array[j] = " ".join(temp_array[j])
            combination = combination.append({'phrase':temp_array[j],\
                                              'target': target, \
                                              'taken': taken}, ignore_index=True)
    
    return combination

## 4. Send the combination to WA

In [ ]:
def evaluation_WA(testing_phrase):
    """
    this function will send the set of combinations to WA and store the following information in a Pandas dataframe:
    phrase, uni-bi-trigram that was taken away, intent detected, intent target, confidence scores
    """
    
    for i in range(len(testing_phrase['phrase'])):
        response = assistant.message(workspace_id=workspace, \
                                     input={'text': testing_phrase['phrase'][i]}, \
                                     alternate_intents= True, user_id= userid)
        dumps = json.dumps(response.get_result(), indent=2)
        data = json.loads(dumps)
        intent1= data['intents'][0]['intent']
        confidence1 = data['intents'][0]['confidence']
        intent2= data['intents'][1]['intent']
        confidence2 = data['intents'][1]['confidence']
        if i == 0:
            df = pd.DataFrame({'phrase':testing_phrase['phrase'][i], \
                               'taken':testing_phrase['taken'][i], \
                               'target':testing_phrase['target'][i], \
                               'intent1': intent1, \
                               'confidence1': confidence1,
                               'intent2': intent2, \
                               'confidence2': confidence2}, index=[0])
        else: 
            df = df.append({'phrase':testing_phrase['phrase'][i],\
                            'taken':testing_phrase['taken'][i], \
                            'target':testing_phrase['target'][i], \
                            'intent1': intent1, \
                            'confidence1': confidence1,
                            'intent2': intent2, \
                            'confidence2': confidence2}, ignore_index=True)
    
    df_pos = df.loc[df["intent1"] == target]
    df_neg = df.loc[df["intent1"] != target]
    
    return df_pos, df_neg
    

    
def visualise_eval(evaluation_pos, evaluation_neg): 
    """
    this function will be able to visualise the pandas dataframe that was create in `evaluation_WA`
    """
    
    fig, (ax1, ax2) = plt.subplots(ncols=2 ,figsize=(17,5))
    sns.set(style="darkgrid")
    ax1.set(ylim=(0, 1))
    ax2.set(ylim=(0, 1))
    
    sns.pointplot(x= evaluation_pos['taken'], y = evaluation_pos['confidence1'], data = evaluation_pos, ax=ax1, color = 'green')
    ax1.set_title("Target Intent was correclty triggered")
    ax1.set_xticklabels(ax1.get_xticklabels(), rotation=40)
    ax1.axhline(threshold, ls='--', color = 'green')
    
   
    sns.pointplot(x= evaluation_neg['taken'], y = evaluation_neg['confidence1'], data = evaluation_neg, ax=ax2, color = 'red')
    ax2.set_title("Failed to trigger the correct intent")
    ax2.set_xticklabels(ax2.get_xticklabels(), rotation=40)
    
    
    return 

### Combine the funcions 

In [ ]:
def unigram_regression(data, target): 
    testing_phrases = create_unigram_set(data, target)
    evaluation_pos, evaluation_neg = evaluation_WA(testing_phrases) 
    visualise_eval(evaluation_pos, evaluation_neg) 
    print("Target Intent was correclty triggered ")
    display(evaluation_pos)
    print("------"*10)
    print("Failed to trigger the correct intent ")
    display(evaluation_neg)
    return 

In [ ]:
def bigram_regression(data, target): 
    testing_phrases = create_bigram_set(data, target)
    evaluation_pos, evaluation_neg = evaluation_WA(testing_phrases) 
    visualise_eval(evaluation_pos, evaluation_neg) 
    print("Target Intent was correclty triggered ")
    display(evaluation_pos)
    print("------"*10)
    print("Failed to trigger the correct intent ")
    display(evaluation_neg)
    return 

In [ ]:
def trigram_regression(data, target): 
    testing_phrases = create_trigram_set(data, target)
    evaluation_pos, evaluation_neg = evaluation_WA(testing_phrases) 
    visualise_eval(evaluation_pos, evaluation_neg) 
    print("Target Intent was correclty triggered ")
    display(evaluation_pos)
    print("------"*10)
    print("Failed to trigger the correct intent ")
    display(evaluation_neg)
    return

## 5. Analysis of the results 

In [ ]:
unigram_regression(data, target)

In [ ]:
bigram_regression(data,target)

In [ ]:
trigram_regression(data,target)

### Author: 
- **[Erika Agostinelli](https://www.linkedin.com/in/erikaagostinelli/)** is a Data Scientist and AI specialist part of IBM Data Science Elite Team where she specializes in Analytics platforms and Machine Learning solutions.

Copyright 2019 IBM Corp. All Rights Reserved.